<a href="https://colab.research.google.com/github/Andyfer004/P2-DS/blob/main/P2_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universidad del Valle de Guatemala  
## Facultad de Ingeniería  
### Departamento de Ciencias de la Computación  


## Proyecto 2 – Análisis Exploratorio de Datos  
**Curso:** CC3084 – Data Science  
**Semestre:** II – 2025  

**Tema seleccionado (Reto 19):** Clasificación de los orígenes de un coágulo de sangre en un accidente cerebrovascular  

**Integrantes del grupo:**  
- Andy Fuentes – 22944  
- Gabriel Paz – 221087  
- Davis Roldán – 22672  
- Jose Marchena – Carné  

**Catedrático:** Prof. Luis Furlán  

**Fecha de entrega:** 14 de septiembre de 2025  


## Introducción

La Ciencia de Datos permite abordar problemas complejos en distintas áreas, incluyendo la medicina.  
En este proyecto se trabajará con datos del **Mayo Clinic STRIP AI Challenge**, cuyo objetivo es clasificar el origen de trombos en pacientes con accidente cerebrovascular isquémico.  

El análisis se centra en diferenciar dos tipos principales de etiología:  
- **Cardioembólico (CE)**: coágulos originados en el corazón.  
- **Ateroesclerosis de arteria grande (LAA)**: coágulos formados en arterias cerebrales principales.  

Un diagnóstico temprano y preciso del origen del coágulo es crucial, ya que permite diseñar **estrategias de prevención secundaria** y tratamientos más efectivos, reduciendo el riesgo de recurrencia del evento.  

Para alcanzar este objetivo, se llevará a cabo un **Análisis Exploratorio de Datos (EDA)** sobre imágenes histopatológicas (Whole Slide Images – WSI), identificando patrones, distribuciones y características relevantes que preparen la base para el modelado posterior.

## Situación problemática

El accidente cerebrovascular isquémico (ACV) es una de las principales causas de discapacidad y muerte a nivel mundial.  
Una parte importante de los ACV se origina en la **formación de coágulos (trombos)** que obstruyen el flujo sanguíneo al cerebro.  
Identificar el **origen del trombo** es crítico:  

- **Cardioembólico (CE):** originado en el corazón, suele requerir anticoagulación.  
- **Ateroesclerosis de arteria grande (LAA):** formado en arterias principales, sugiere tratamiento antiagregante y control de factores de riesgo.  

El reto radica en que, aun después de estudios clínicos e imagenológicos, el **origen del trombo muchas veces queda indeterminado**, lo que dificulta aplicar la estrategia de prevención más adecuada.


## Problema científico

¿Es posible identificar **patrones histológicos cuantificables** en imágenes completas de trombos (Whole Slide Images) que permitan a un modelo de Ciencia de Datos **clasificar de forma confiable** su origen en **CE o LAA**?


## Objetivos

**Objetivo general**  
Desarrollar un análisis exploratorio de datos (EDA) sobre las imágenes histopatológicas de trombos, con el fin de identificar patrones y características relevantes que preparen la base para un modelo de clasificación CE vs LAA.

**Objetivos específicos**  
1. Describir la estructura del dataset, incluyendo número de casos, distribución por clase y metadatos asociados.  
2. Implementar técnicas de preprocesamiento para filtrar tiles no informativos (fondo, blur, bajo tejido) y normalizar la tinción.  
3. Explorar métricas básicas de color, textura y cobertura de tejido para cada clase.  
4. Visualizar los datos mediante histogramas, boxplots y reducciones de dimensionalidad (UMAP/t-SNE) para detectar posibles separaciones entre clases.

## Investigación preliminar

### Contexto médico
El **accidente cerebrovascular isquémico (ACV)** ocurre cuando un coágulo interrumpe el flujo sanguíneo al cerebro.  
Existen varias causas, pero dos de las más frecuentes son:  

- **Cardioembólico (CE):** coágulos que se originan en el corazón, por ejemplo a raíz de fibrilación auricular o insuficiencia cardíaca.  
- **Ateroesclerosis de arteria grande (LAA):** trombos formados en arterias cerebrales principales debido a placas ateroscleróticas.  

La distinción entre ambos es esencial, ya que determina el tratamiento: anticoagulación para CE vs. antiagregación y control de factores de riesgo para LAA.  

### Imágenes histopatológicas
En el reto se utilizan **Whole Slide Images (WSI)** de trombos extraídos mediante trombectomía.  
Estas imágenes se obtienen con tinción **Hematoxilina & Eosina (H&E)**, que resalta estructuras celulares:  
- Hematoxilina → tiñe núcleos de color azul/púrpura.  
- Eosina → tiñe citoplasma y componentes extracelulares de rosado.  

### Relevancia del análisis computacional
El análisis manual de WSI es complejo y requiere mucho tiempo de especialistas.  
Aquí entra la **Ciencia de Datos**, que permite:  
- Fragmentar imágenes en **tiles** para su estudio.  
- Aplicar técnicas de **preprocesamiento** como normalización de color y eliminación de ruido.  
- Extraer **características cuantitativas** (textura, color, cobertura de tejido).  
- Generar representaciones que faciliten la clasificación con modelos de Visión por Computadora.

### Estado del arte en Visión Artificial para WSI
- Se emplean **CNNs** preentrenadas (ResNet, EfficientNet) para extraer embeddings.  
- Se utilizan técnicas de **Multiple Instance Learning (MIL)** donde cada WSI se representa como conjunto de tiles.  
- Métodos recientes incluyen **transformers** y mecanismos de **atención**, que aprenden qué regiones de la WSI son más informativas para predecir el origen del trombo.  

En este proyecto, no se busca aún entrenar un modelo complejo, sino realizar un **análisis exploratorio** que evidencie patrones básicos y prepare los datos para etapas posteriores.